# Data Collection

We follow the same procedure for the import data as the export data by selecting the appropriate division elements for import using the link for the import data.

<h2> Importing libraries for Webscraping the Import data </h2>

In [12]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import pandas as pd
import html5lib

In [13]:
#!pip install webdriver-manager

In [14]:
import_comm_url = "https://commerce-app.gov.in/eidb/icntcomq.asp"

# setting up the driver
chrome_options = Options()
chrome_options.headless = True
from webdriver_manager.chrome import ChromeDriverManager

get_country_list = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)

get_country_list.get(import_comm_url)

# Populating country list
country_names = []
for o in Select(get_country_list.find_element_by_name('cntcode')).options:
          country_names.append(o.text)
        
get_country_list.close()


Looking for [chromedriver 78.0.3904.105 mac64] driver in cache 
File found in cache by path [/Users/anjaliagrawal/.wdm/drivers/chromedriver/78.0.3904.105/mac64/chromedriver]


In [15]:
import_comm_data = pd.DataFrame()

In [20]:
# Scraping commodity data
def get_import_comm(year, upper_limit=len(country_names)):
    
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
    driver.get(import_comm_url)
    
    global import_comm_data    
    
    for ix in range(0, upper_limit):
        country_name = country_names[ix]
        
        print(year, country_name)
        
        Select(driver.find_element_by_id('select2')).select_by_value(str(year)) # Select year

        Select(driver.find_element_by_id('select3')).select_by_visible_text(country_name) # Select country name

        # Additional options    
        driver.find_element_by_id('radioDAll').click()
        driver.find_element_by_id('radiousd').click()

        driver.find_element_by_id('button1').click()

        try:
            tbl = driver.find_element_by_tag_name('table').get_attribute('outerHTML')
            df = pd.read_html(tbl)[0]

            # Scraping dataframe
            df['country'] = country_name
            df['year'] = year
            df.rename(columns={df.columns[4]:'value'}, inplace=True) # Renaming column to use in append


            import_comm_data = import_comm_data.append(df.iloc[:-3,[1,2,4,6,7]], sort=False)
            
        except:
            pass
        
        driver.get(import_comm_url)
    
    driver.close()

In [26]:
for year in range(1997, 2020):
    get_import_comm(year)


Looking for [chromedriver 78.0.3904.105 mac64] driver in cache 
File found in cache by path [/Users/anjaliagrawal/.wdm/drivers/chromedriver/78.0.3904.105/mac64/chromedriver]
1997 AFGHANISTAN TIS
1997 ALBANIA
1997 ALGERIA
1997 AMERI SAMOA
1997 ANDORRA
1997 ANGOLA
1997 ANGUILLA
1997 ANTARTICA
1997 ANTIGUA
1997 ARGENTINA
1997 ARMENIA
1997 ARUBA
1997 AUSTRALIA
1997 AUSTRIA
1997 AZERBAIJAN
1997 BAHAMAS
1997 BAHARAIN IS
1997 BANGLADESH PR
1997 BARBADOS
1997 BELARUS
1997 BELGIUM
1997 BELIZE
1997 BENIN
1997 BERMUDA
1997 BHUTAN
1997 BOLIVIA
1997 BOSNIA-HRZGOVIN
1997 BOTSWANA
1997 BR VIRGN IS
1997 BRAZIL
1997 BRUNEI
1997 BULGARIA
1997 BURKINA FASO
1997 BURUNDI
1997 C AFRI REP
1997 CAMBODIA
1997 CAMEROON
1997 CANADA
1997 CANARY IS
1997 CAPE VERDE IS
1997 CAYMAN IS
1997 CHAD
1997 CHANNEL IS
1997 CHILE
1997 CHINA P RP
1997 CHRISTMAS IS.
1997 COCOS IS
1997 COLOMBIA
1997 COMOROS
1997 CONGO D. REP.
1997 CONGO P REP
1997 COOK IS
1997 COSTA RICA
1997 COTE D' IVOIRE
1997 CROATIA
1997 CUBA
1997 CURACAO
1

In [27]:
import_comm_data.to_csv('all_import.csv')